# **Investigando padrões de homicídio no Brasil com dados do SIM/DATASUS**

Este notebook explora dados públicos de mortalidade do SIM/DATASUS para investigar possíveis padrões anômalos ou repetitivos em homicídios, com o objetivo de identificar comportamentos semelhantes a serial killers ou clusters incomuns de violência.

🔍 Etapas abordadas:
- Conversão e pré-processamento dos dados (.dbc → .dbf → .csv)
- Enriquecimento com informações geográficas (IBGE)
- Análise exploratória por perfil de vítima
- Clusterização não supervisionada com KMeans


*Por Lucio Nunes*

# **Carregamento e Conversão de Dados**

In [ ]:
# Acessar arquivos do google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install dbfread
!pip install dbf

In [ ]:
import os

path = "/content/drive/MyDrive/KillerData/db"
arquivos = os.listdir(path)
print(arquivos)

['tb_municip.dbf', 'DOBR2022.dbc', 'DOBR2022.dbf', 'DOBR2022.csv', 'df_hour.csv', 'lista_canonicos.csv', 'DOBR2022_serial.csv']


In [ ]:
from dbfread import DBF
import pandas as pd


homicidios = []

table = DBF(path, encoding='latin1', load=False)

for record in table:
    cid = record.get('CAUSABAS', '')
    if cid.startswith(('X8', 'X9', 'Y0')):  # CID X85–Y09
        homicidios.append(record)

df = pd.DataFrame(homicidios)

"\nhomicidios = []\n\ntable = DBF(path, encoding='latin1', load=False)\n\nfor record in table:\n    cid = record.get('CAUSABAS', '')\n    if cid.startswith(('X8', 'X9', 'Y0')):  # CID X85–Y09\n        homicidios.append(record)\n\ndf = pd.DataFrame(homicidios)\ndf.head()\n"

In [ ]:
df.head()

In [ ]:
# Salvar o df em csv
df.to_csv('/content/drive/MyDrive/KillerData/db/DOBR2022.csv', index=False)

In [ ]:
import pandas as pd

# Ler csv
path = '/content/drive/MyDrive/KillerData/db/DOBR2022.csv'
df = pd.read_csv(path)

/tmp/ipython-input-57-527691750.py:5: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
df.head()

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
0,1,2,22012022,2102.0,828.0,280590.0,23121977.0,444,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170
1,1,2,23012022,430.0,828.0,280140.0,22062001.0,420,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171
2,1,2,22052022,1710.0,835.0,353240.0,30102020.0,401,1,1.0,...,SXXXSX,NaN,NaN,NaN,2062022.0,3.0,2062022.0,NaN,2.0,177
3,1,2,27012022,1020.0,828.0,280210.0,7121989.0,432,2,4.0,...,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,210
4,1,2,27012022,530.0,828.0,280330.0,15041976.0,445,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212


In [ ]:
# Visualizar atributos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45482 entries, 0 to 45481
Data columns (total 87 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ORIGEM      45482 non-null  int64  
 1   TIPOBITO    45482 non-null  int64  
 2   DTOBITO     45482 non-null  int64  
 3   HORAOBITO   37270 non-null  float64
 4   NATURAL     43786 non-null  float64
 5   CODMUNNATU  43421 non-null  float64
 6   DTNASC      44791 non-null  float64
 7   IDADE       45482 non-null  int64  
 8   SEXO        45482 non-null  int64  
 9   RACACOR     44823 non-null  float64
 10  ESTCIV      43931 non-null  float64
 11  ESC         42684 non-null  float64
 12  ESC2010     42487 non-null  float64
 13  SERIESCFAL  15005 non-null  float64
 14  OCUP        35325 non-null  float64
 15  CODMUNRES   45482 non-null  int64  
 16  LOCOCOR     45482 non-null  int64  
 17  CODESTAB    10353 non-null  float64
 18  ESTABDESCR  0 non-null      float64
 19  CODMUNOCOR  45482 non-nul

In [ ]:
# Selecionar as colunas relevantes
colunas_analise_serial = [
    'DTOBITO', 'HORAOBITO', 'IDADE', 'SEXO', 'RACACOR',
    'ESC2010', 'OCUP', 'CODMUNRES', 'CODMUNOCOR', 'LOCOCOR', 'CAUSABAS'
]

df_serial = df[colunas_analise_serial].copy()

In [ ]:
df_serial.head()

,DTOBITO,HORAOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS
0,22012022,2102.0,444,1,4.0,2.0,768110.0,280480,280030,1,X954
1,23012022,430.0,420,1,4.0,3.0,622020.0,280140,280140,3,X994
2,22052022,1710.0,401,1,1.0,NaN,NaN,350710,350410,1,Y040
3,27012022,1020.0,432,2,4.0,2.0,622020.0,280750,280210,1,X994
4,27012022,530.0,445,1,4.0,2.0,354705.0,280130,280130,3,X950


In [ ]:
save_path = '/content/drive/MyDrive/KillerData/db/DOBR2022_serial.csv'

# Salvar o DataFrame como CSV
df_serial.to_csv(save_path, index=False, encoding='latin1')

print("Arquivo salvo com sucesso em:", save_path)

'\nsave_path = \'/content/drive/MyDrive/KillerData/db/DOBR2022_serial.csv\'\n\n# Salvar o DataFrame como CSV\ndf_serial.to_csv(save_path, index=False, encoding=\'latin1\')\n\nprint("Arquivo salvo com sucesso em:", save_path)\n'

# **Carregar CSV**

In [ ]:
import pandas as pd

table_path = '/content/drive/MyDrive/KillerData/db/DOBR2022_serial.csv'

df_serial = pd.read_csv(table_path)

In [ ]:
df_serial.head()

,DTOBITO,HORAOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS
0,22012022,2102.0,444,1,4.0,2.0,768110.0,280480,280030,1,X954
1,23012022,430.0,420,1,4.0,3.0,622020.0,280140,280140,3,X994
2,22052022,1710.0,401,1,1.0,NaN,NaN,350710,350410,1,Y040
3,27012022,1020.0,432,2,4.0,2.0,622020.0,280750,280210,1,X994
4,27012022,530.0,445,1,4.0,2.0,354705.0,280130,280130,3,X950


# **Pré-processamento e Limpeza**

In [ ]:
# Verificar quantidade de valores nulos por coluna
df_serial.isnull().sum().sort_values(ascending=False)

,0
OCUP,10157
HORAOBITO,8212
ESC2010,2995
RACACOR,659
DTOBITO,0
IDADE,0
SEXO,0
CODMUNRES,0
CODMUNOCOR,0
LOCOCOR,0


In [ ]:
# Preencher OCUP (Ocupação) como não informado para não perder 22% dos dados da db
# Há risco de gerar víes se os nulos estiverem concentrados em um perfil de vítima

df_serial['OCUP'] = df_serial['OCUP'].fillna(-1)
df_serial['OCUP'] = df_serial['OCUP'].astype(int).astype(str)

In [ ]:
# Separar HORAOBITO por faixas de horário e gerar desconhecido para valores nulos

# Arrumar coluna de horas
import datetime

# Função para converter float tipo 2102.0 em time(21, 2)
def converter_hora(h):
    if pd.isna(h):
        return None
    h = int(h)
    h_str = f"{h:04d}"  # garante 4 dígitos, ex: 430 → '0430'
    hora = int(h_str[:2])
    minuto = int(h_str[2:])
    return datetime.time(hour=hora, minute=minuto)

def hora_hhmm_para_decimal(h):
    if pd.isna(h):
        return None
    h = int(h)
    h_str = f"{h:04d}"  # ex: 430 → '0430'
    horas = int(h_str[:2])
    minutos = int(h_str[2:])
    return horas + minutos / 60

df_serial['HORAOBITO_DECIMAL'] = df_serial['HORAOBITO'].apply(hora_hhmm_para_decimal)

# Aplicar no dataframe
df_serial['HORAOBITO_REAL'] = df_serial['HORAOBITO'].apply(converter_hora)

# Corrigir df_serial
df_serial['HORAOBITO_CAT'] = pd.cut(df_serial['HORAOBITO_DECIMAL'],
                                    bins=[0, 6, 12, 18, 24],
                                    labels=['Madrugada', 'Manhã', 'Tarde', 'Noite'],
                                    include_lowest=True)

df_serial['HORAOBITO_CAT'] = df_serial['HORAOBITO_CAT'].cat.add_categories('Desconhecido')
df_serial['HORAOBITO_CAT'] = df_serial['HORAOBITO_CAT'].fillna('Desconhecido')

In [ ]:
df_serial.head()

,DTOBITO,HORAOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_DECIMAL,HORAOBITO_REAL,HORAOBITO_CAT
0,22012022,2102.0,444,1,4.0,2.0,768110,280480,280030,1,X954,21.033333,21:02:00,Noite
1,23012022,430.0,420,1,4.0,3.0,622020,280140,280140,3,X994,4.500000,04:30:00,Madrugada
2,22052022,1710.0,401,1,1.0,NaN,-1,350710,350410,1,Y040,17.166667,17:10:00,Tarde
3,27012022,1020.0,432,2,4.0,2.0,622020,280750,280210,1,X994,10.333333,10:20:00,Manhã
4,27012022,530.0,445,1,4.0,2.0,354705,280130,280130,3,X950,5.500000,05:30:00,Madrugada


In [ ]:
# Gerar id no df
df_serial.reset_index(drop=True, inplace=True)
df_serial.insert(0, 'ID', df_serial.index + 1)

In [ ]:
df_serial.head()

,ID,DTOBITO,HORAOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_DECIMAL,HORAOBITO_REAL,HORAOBITO_CAT
0,1,22012022,2102.0,444,1,4.0,2.0,768110,280480,280030,1,X954,21.033333,21:02:00,Noite
1,2,23012022,430.0,420,1,4.0,3.0,622020,280140,280140,3,X994,4.500000,04:30:00,Madrugada
2,3,22052022,1710.0,401,1,1.0,NaN,-1,350710,350410,1,Y040,17.166667,17:10:00,Tarde
3,4,27012022,1020.0,432,2,4.0,2.0,622020,280750,280210,1,X994,10.333333,10:20:00,Manhã
4,5,27012022,530.0,445,1,4.0,2.0,354705,280130,280130,3,X950,5.500000,05:30:00,Madrugada


In [ ]:
# Gerar um df_hora com 'ID' e 'HORAOBITO_REAL'
df_hour = df_serial[['ID', 'HORAOBITO_REAL']].copy()
print(df_hour.head())
df_hour.info()

   ID HORAOBITO_REAL
0   1       21:02:00
1   2       04:30:00
2   3       17:10:00
3   4       10:20:00
4   5       05:30:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45482 entries, 0 to 45481
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              45482 non-null  int64 
 1   HORAOBITO_REAL  37270 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.8+ KB


In [ ]:
# Salvar df_hour
save_path_hour = '/content/drive/MyDrive/KillerData/db/df_hour.csv'

# Salvar o DataFrame como CSV
df_hour.to_csv(save_path_hour, index=False)

print("Arquivo df_hour salvo com sucesso em:", save_path_hour)

'\n# Salvar df_hour\nsave_path_hour = \'/content/drive/MyDrive/KillerData/db/df_hour.csv\'\n\n# Salvar o DataFrame como CSV\ndf_hour.to_csv(save_path_hour, index=False)\n\nprint("Arquivo df_hour salvo com sucesso em:", save_path_hour)\n'

In [ ]:
# Remover colunas 'HORAOBITO_REAL', 'HORAOBITO_DECIMAL' e 'HORA_OBITO' do df_serial

df_serial = df_serial.drop(columns=['HORAOBITO_REAL', 'HORAOBITO_DECIMAL', 'HORAOBITO'])

In [ ]:
df_serial.head()

,ID,DTOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT
0,1,22012022,444,1,4.0,2.0,768110,280480,280030,1,X954,Noite
1,2,23012022,420,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada
2,3,22052022,401,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde
3,4,27012022,432,2,4.0,2.0,622020,280750,280210,1,X994,Manhã
4,5,27012022,445,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada


In [ ]:
# Arrumar coluna idades
"""
Sistema DATASUS usa:
1XX → idade em horas
2XX → idade em dias
3XX → idade em meses
4XX → idade em anos
"""

def decodificar_idade(cod):
    if pd.isna(cod):
        return None
    cod = int(cod)
    unidade = int(str(cod)[0])
    valor = int(str(cod)[1:])
    if unidade == 1:
        return round(valor / 24 / 365, 2)  # horas → anos
    elif unidade == 2:
        return round(valor / 365, 2)       # dias → anos
    elif unidade == 3:
        return round(valor / 12, 2)        # meses → anos
    elif unidade == 4:
        return valor                       # anos
    else:
        return None

df_serial['IDADE_ANOS'] = df_serial['IDADE'].apply(decodificar_idade)

In [ ]:
df_serial.head()

,ID,DTOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS
0,1,22012022,444,1,4.0,2.0,768110,280480,280030,1,X954,Noite,44.0
1,2,23012022,420,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada,20.0
2,3,22052022,401,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde,1.0
3,4,27012022,432,2,4.0,2.0,622020,280750,280210,1,X994,Manhã,32.0
4,5,27012022,445,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada,45.0


In [ ]:
# Trabalhar com faixas etárias (Serial Killer não mata por idade exata)
bins = [0, 5, 12, 18, 30, 45, 60, 80, 150]
labels = [
    '0-4',
    '5-12',
    '13-17',
    '18-29',
    '30-44',
    '45-59',
    '60-79',
    '80+'
]

df_serial['FAIXA_ETARIA'] = pd.cut(df_serial['IDADE_ANOS'], bins=bins, labels=labels, right=False)

In [ ]:
df_serial.head()

,ID,DTOBITO,IDADE,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA
0,1,22012022,444,1,4.0,2.0,768110,280480,280030,1,X954,Noite,44.0,30-44
1,2,23012022,420,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada,20.0,18-29
2,3,22052022,401,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde,1.0,0-4
3,4,27012022,432,2,4.0,2.0,622020,280750,280210,1,X994,Manhã,32.0,30-44
4,5,27012022,445,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada,45.0,45-59


In [ ]:
df_serial = df_serial.drop(columns=['IDADE'])

In [ ]:
df_serial.head()

,ID,DTOBITO,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA
0,1,22012022,1,4.0,2.0,768110,280480,280030,1,X954,Noite,44.0,30-44
1,2,23012022,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada,20.0,18-29
2,3,22052022,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde,1.0,0-4
3,4,27012022,2,4.0,2.0,622020,280750,280210,1,X994,Manhã,32.0,30-44
4,5,27012022,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada,45.0,45-59


In [ ]:
# Converter local de ocorrência

locais = {
    1: 'Hospital',
    2: 'Domicílio',
    3: 'Via pública',
    4: 'Outros',
    5: 'Ignorado',
    6: 'Outro serviço de saúde',
    9: 'Sem informação'
}

df_serial['LOCOCOR_DESC'] = df_serial['LOCOCOR'].map(locais)

In [ ]:
# Extrair dia e mês da coluna DTOBITO
df_serial['DTOBITO'] = df_serial['DTOBITO'].astype(str).str.zfill(8)
df_serial['DIAOBITO'] = df_serial['DTOBITO'].str[:2].astype(int)
df_serial['MESOBITO'] = df_serial['DTOBITO'].str[2:4].astype(int)

# Remover a coluna original
df_serial.drop(columns=['DTOBITO'], inplace=True)

In [ ]:
df_serial.head()

,ID,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA,LOCOCOR_DESC,DIAOBITO,MESOBITO
0,1,1,4.0,2.0,768110,280480,280030,1,X954,Noite,44.0,30-44,Hospital,22,1
1,2,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada,20.0,18-29,Via pública,23,1
2,3,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde,1.0,0-4,Hospital,22,5
3,4,2,4.0,2.0,622020,280750,280210,1,X994,Manhã,32.0,30-44,Hospital,27,1
4,5,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada,45.0,45-59,Via pública,27,1


In [ ]:
# Tratar trabalhos

ocup_table = '/content/drive/MyDrive/KillerData/db/lista_canonicos.csv'
ocup = pd.read_csv(ocup_table, encoding='latin1')

,codigo,termo
0,8485-05,Abatedor
1,7663-05,Acabador de embalagens (flexÃ­veis e cartotÃ©c...
2,7161-05,Acabador de superfÃ­cies de concreto
3,8485-10,AÃ§ougueiro
4,3762-05,Acrobata


In [ ]:
ocup.head()

In [ ]:
# Converter para inteiro, garantir 6 dígitos com zero à esquerda e inserir traço
df_serial['OCUP_FORMATADA'] = df_serial['OCUP'].dropna().astype(int).astype(str).str.zfill(6).str.replace(r'(\d{4})(\d{2})', r'\1-\2', regex=True)

# Preencher valores nulos com 'Desconhecido'
df_serial['OCUP_FORMATADA'] = df_serial['OCUP_FORMATADA'].fillna('Desconhecido')

In [ ]:
df_serial.head()

,ID,SEXO,RACACOR,ESC2010,OCUP,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA,LOCOCOR_DESC,DIAOBITO,MESOBITO,OCUP_FORMATADA
0,1,1,4.0,2.0,768110,280480,280030,1,X954,Noite,44.0,30-44,Hospital,22,1,7681-10
1,2,1,4.0,3.0,622020,280140,280140,3,X994,Madrugada,20.0,18-29,Via pública,23,1,6220-20
2,3,1,1.0,NaN,-1,350710,350410,1,Y040,Tarde,1.0,0-4,Hospital,22,5,-00001
3,4,2,4.0,2.0,622020,280750,280210,1,X994,Manhã,32.0,30-44,Hospital,27,1,6220-20
4,5,1,4.0,2.0,354705,280130,280130,3,X950,Madrugada,45.0,45-59,Via pública,27,1,3547-05


In [ ]:
# Fazer merge com a tabela de ocupações
df_serial = df_serial.merge(ocup[['codigo', 'termo']], left_on='OCUP_FORMATADA', right_on='codigo', how='left')

# Renomear a coluna para melhor interpretação
df_serial.rename(columns={'termo': 'OCUP_DESC'}, inplace=True)

# Exibir os primeiros exemplos
df_serial[['OCUP', 'OCUP_FORMATADA', 'OCUP_DESC']].drop_duplicates().head(10)

,OCUP,OCUP_FORMATADA,OCUP_DESC
0,768110,7681-10,TecelÃ£o de tapetes; a mÃ£o
1,622020,6220-20,Trabalhador volante da agricultura
2,-1,-00001,NaN
4,354705,3547-05,Representante comercial autÃ´nomo
5,717020,7170-20,Servente de obras
6,723310,7233-10,Pintor a pincel e rolo (exceto obras e estrutu...
7,621005,6210-05,Trabalhador agropecuÃ¡rio em geral
10,512105,5121-05,Empregado domÃ©stico nos serviÃ§os gerais
11,141615,1416-15,Gerente de logÃ­stica (armazenagem e distribui...
12,991305,9913-05,Funileiro de veÃ­culos (reparaÃ§Ã£o)


In [ ]:
df_serial.drop(columns=['OCUP_FORMATADA', 'codigo', 'OCUP'], inplace=True)

,ID,SEXO,RACACOR,ESC2010,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA,LOCOCOR_DESC,DIAOBITO,MESOBITO,OCUP_DESC
0,1,1,4.0,2.0,280480,280030,1,X954,Noite,44.0,30-44,Hospital,22,1,TecelÃ£o de tapetes; a mÃ£o
1,2,1,4.0,3.0,280140,280140,3,X994,Madrugada,20.0,18-29,Via pública,23,1,Trabalhador volante da agricultura
2,3,1,1.0,NaN,350710,350410,1,Y040,Tarde,1.0,0-4,Hospital,22,5,NaN
3,4,2,4.0,2.0,280750,280210,1,X994,Manhã,32.0,30-44,Hospital,27,1,Trabalhador volante da agricultura
4,5,1,4.0,2.0,280130,280130,3,X950,Madrugada,45.0,45-59,Via pública,27,1,Representante comercial autÃ´nomo


In [ ]:
# Corrigir problemas de codificação na coluna de descrição das ocupações
df_serial['OCUP_DESC'] = df_serial['OCUP_DESC'].astype(str).apply(lambda x: x.encode('latin1').decode('utf-8') if x != 'nan' else x)

In [ ]:
df_serial.head()

,ID,SEXO,RACACOR,ESC2010,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA,LOCOCOR_DESC,DIAOBITO,MESOBITO,OCUP_DESC
0,1,1,4.0,2.0,280480,280030,1,X954,Noite,44.0,30-44,Hospital,22,1,Tecelão de tapetes; a mão
1,2,1,4.0,3.0,280140,280140,3,X994,Madrugada,20.0,18-29,Via pública,23,1,Trabalhador volante da agricultura
2,3,1,1.0,NaN,350710,350410,1,Y040,Tarde,1.0,0-4,Hospital,22,5,nan
3,4,2,4.0,2.0,280750,280210,1,X994,Manhã,32.0,30-44,Hospital,27,1,Trabalhador volante da agricultura
4,5,1,4.0,2.0,280130,280130,3,X950,Madrugada,45.0,45-59,Via pública,27,1,Representante comercial autônomo


In [ ]:
# Remover todas as instâncias com local de ocorrência 'Hospital', pois é local monitorado
df_serial = df_serial[df_serial['LOCOCOR_DESC'] != 'Hospital']

In [ ]:
# Converter CODMUNOCOR para coordenadas para mapear por raios de distância (ou regiões)

municipios_path = '/content/drive/MyDrive/KillerData/db/tb_municip.dbf'

In [ ]:
# Ler dbf

table_municipios = DBF(municipios_path, encoding='latin1', load=False)

municipios_list = []
for record in table_municipios:
    municipios_list.append(record)

df_municipios = pd.DataFrame(municipios_list)
df_municipios.head()

,CO_MUNICIP,CO_REGIONA,SG_UF,CO_MACRORR,CO_MESORRE,CO_MICRORR,CO_UF_IBGE,NO_MUNICIP,SG_MUNICIP,NU_DDD,...,ST_PAN,ST_PACTO_R,CO_MUNICI9,TP_TIPOLOG,CO_GRUPO_P,ST_CNES,CO_REGIAO3,ST_PACTO,ST_CIB_SAS,DT_MANUTEN
0,261080,2606,PE,2606,2603,26007,26,PEDRA,,87,...,N,N,124,BAIXA RENDA,03,N,138,,,
1,314890,3115,MG,3105,3109,31044,31,PEDRA DO INDAIA,,37,...,N,N,572,ESTAGNADA,03,N,592,,,
2,240950,2403,RN,2402,2404,24016,24,PEDRA GRANDE,,84,...,N,N,102,BAIXA RENDA,03,N,84,,,
3,314910,3110,MG,3101,3110,31053,31,PEDRALVA,,35,...,N,N,574,ESTAGNADA,03,N,630,,,
4,292410,2903,BA,2914,2903,29012,29,PEDRAO,,75,...,N,N,296,ESTAGNADA,03,N,400,,,


In [ ]:
colunas_municipios = ['CO_MUNICIP', 'CO_MESORRE', 'NO_MUNICIP']
df_municipios_small = df_municipios[colunas_municipios].copy()
df_municipios_small.head()

,CO_MUNICIP,CO_MESORRE,NO_MUNICIP
0,261080,2603,PEDRA
1,314890,3109,PEDRA DO INDAIA
2,240950,2404,PEDRA GRANDE
3,314910,3110,PEDRALVA
4,292410,2903,PEDRAO


In [ ]:
# Juntar tabelas 'df_serial' and 'df_municipios_small' by getting correspondence between 'CODMUNOCOR' and 'CO_MUNICIP'
# Converter colunas
df_serial['CODMUNOCOR'] = df_serial['CODMUNOCOR'].astype(str)
df_municipios_small['CO_MUNICIP'] = df_municipios_small['CO_MUNICIP'].astype(str)

df_merged = pd.merge(df_serial, df_municipios_small, left_on='CODMUNOCOR', right_on='CO_MUNICIP', how='left')

,ID,SEXO,RACACOR,ESC2010,CODMUNRES,CODMUNOCOR,LOCOCOR,CAUSABAS,HORAOBITO_CAT,IDADE_ANOS,FAIXA_ETARIA,LOCOCOR_DESC,DIAOBITO,MESOBITO,OCUP_DESC,CO_MUNICIP,CO_MESORRE,NO_MUNICIP
0,2,1,4.0,3.0,280140,280140,3,X994,Madrugada,20.0,18-29,Via pública,23,1,Trabalhador volante da agricultura,280140,2801,CARIRA
1,5,1,4.0,2.0,280130,280130,3,X950,Madrugada,45.0,45-59,Via pública,27,1,Representante comercial autônomo,280130,2803,CAPELA
2,6,1,4.0,1.0,280480,280480,4,X954,Noite,29.0,18-29,Outros,26,1,Servente de obras,280480,2803,NOSSA SENHORA DO SOCORRO
3,8,1,4.0,1.0,220995,220800,5,Y099,Tarde,50.0,45-59,Ignorado,28,5,Trabalhador agropecuário em geral,220800,2204,PICOS
4,10,1,2.0,0.0,500325,500325,4,X997,Manhã,19.0,18-29,Outros,12,5,nan,500325,5003,COSTA RICA


In [ ]:
df_merged.head()

# **Análises Exploratórias**

In [ ]:
df_merged.groupby(['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'SEXO', 'FAIXA_ETARIA']).size().sort_values(ascending=False).head(20)

/tmp/ipython-input-38-3046871407.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_merged.groupby(['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'SEXO', 'FAIXA_ETARIA']).size().sort_values(ascending=False).head(20)


NO_MUNICIP               HORAOBITO_CAT  CAUSABAS  SEXO  FAIXA_ETARIA
RECIFE                   Desconhecido   X954      1     18-29           132
MANAUS                   Noite          X954      1     18-29           120
SALVADOR                 Noite          X954      1     18-29           101
                         Madrugada      X954      1     18-29            86
FORTALEZA                Noite          X954      1     18-29            85
                         Tarde          X954      1     18-29            84
RECIFE                   Desconhecido   X954      1     30-44            80
SALVADOR                 Tarde          X954      1     18-29            79
RIO DE JANEIRO           Desconhecido   X954      1     18-29            77
JABOATAO DOS GUARARAPES  Desconhecido   X954      1     18-29            67
MANAUS                   Tarde          X954      1     18-29            64
FORTALEZA                Madrugada      X954      1     18-29            60
FEIRA DE SANTANA         Noite          X959      1     18-29            60
FORTALEZA                Noite          X954      1     30-44            59
RIO DE JANEIRO           Desconhecido   X954      1     30-44            56
SALVADOR                 Manhã          X954      1     18-29            53
CABO DE SANTO AGOSTINHO  Desconhecido   X954      1     18-29            52
VILA VELHA               Desconhecido   X954      1     18-29            49
BELO HORIZONTE           Desconhecido   X954      1     18-29            49
FEIRA DE SANTANA         Tarde          X959      1     18-29            47
dtype: int64

In [ ]:
# Encontrado um mapa de cidades mais violentas do país

In [ ]:
df_merged.groupby('NO_MUNICIP').size().sort_values(ascending=False).head(20)

,0
NO_MUNICIP,
SALVADOR,879
MANAUS,782
FORTALEZA,735
RIO DE JANEIRO,573
RECIFE,463
MACEIO,335
FEIRA DE SANTANA,320
PORTO ALEGRE,308
TERESINA,293


In [ ]:
# Agrupar adicionando mês e cidade para detectar surtos
df_merged.groupby(['MESOBITO', 'NO_MUNICIP', 'CAUSABAS', 'FAIXA_ETARIA', 'SEXO']).size().sort_values(ascending=False).head(20)

/tmp/ipython-input-99-4247089367.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_merged.groupby(['MESOBITO', 'NO_MUNICIP', 'CAUSABAS', 'FAIXA_ETARIA', 'SEXO']).size().sort_values(ascending=False).head(20)


In [ ]:
# Agrupar por padrão de vítima e ocorrência
agrupado = df_merged.groupby(['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'SEXO', 'FAIXA_ETARIA']).size().reset_index(name='quantidade')

# Manter só os grupos que ocorreram pelo menos 3 vezes
sospeitos = agrupado[agrupado['quantidade'] >= 3]

# Total de registros por cidade
totais_por_municipio = df_merged['NO_MUNICIP'].value_counts()

# Definir cidades pequenas (< 100 registros)
cidades_pequenas = totais_por_municipio[totais_por_municipio < 100].index

# Filtrar os padrões suspeitos que ocorrem nessas cidades pequenas
sospeitos_pequenos = sospeitos[sospeitos['NO_MUNICIP'].isin(cidades_pequenas)]

# Exibir resultado
sospeitos_pequenos.sort_values(by='quantidade', ascending=False)

In [ ]:
# Definir faixas etárias 40+
faixas_40plus = ['45-59 anos', '60-79 anos', '80+ anos']

# Filtrar mulheres 40+ fora do padrão típico
perfil_fora_do_padrao = df_merged[
    (df_merged['SEXO'] == 2) &
    (df_merged['FAIXA_ETARIA'].isin(faixas_40plus))
]

# Agrupar por características combinadas
agrupado = perfil_fora_do_padrao.groupby(
    ['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'FAIXA_ETARIA']
).size().reset_index(name='quantidade')

# Filtrar agrupamentos repetidos
sinais_suspeitos = agrupado[agrupado['quantidade'] >= 2]

# Mostrar o resultado
sinais_suspeitos.sort_values(by='quantidade', ascending=False)


In [ ]:
# Filtrar vítimas do sexo feminino entre 13 e 17 anos
df_meninas = df_merged[
    (df_merged['SEXO'] == 2) &
    (df_merged['FAIXA_ETARIA'] == '13-17 anos')
]

# Agrupar por cidade, hora, causa e faixa etária
agrupado_meninas = df_meninas.groupby(
    ['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'FAIXA_ETARIA']
).size().reset_index(name='quantidade')

# Pegar apenas agrupamentos repetidos (2 ou mais)
padroes_meninas = agrupado_meninas[agrupado_meninas['quantidade'] >= 2]

# Exibir ordenado por quantidade
padroes_meninas.sort_values(by='quantidade', ascending=False)

In [ ]:
# Definir causas incomuns associadas a serialidade (não arma de fogo)
causas_incomuns = ['X70', 'X71', 'X84', 'X85', 'X90', 'X91', 'X92', 'Y09', 'Y10', 'Y16', 'Y19']

# Filtrar apenas essas causas no dataset
df_incomuns = df_merged[df_merged['CAUSABAS'].isin(causas_incomuns)]

# Agrupar por cidade, hora, causa, sexo e faixa etária
agrupado_incomuns = df_incomuns.groupby(
    ['NO_MUNICIP', 'HORAOBITO_CAT', 'CAUSABAS', 'SEXO', 'FAIXA_ETARIA']
).size().reset_index(name='quantidade')

# Filtrar agrupamentos com pelo menos 2 ocorrências idênticas
padroes_incomuns = agrupado_incomuns[agrupado_incomuns['quantidade'] >= 2]

# Exibir resultados ordenados
padroes_incomuns.sort_values(by='quantidade', ascending=False)

# **Agrupamento com KMeans**

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import pandas as pd

# Subconjunto de colunas para clusterização
variaveis = ['IDADE_ANOS', 'SEXO', 'HORAOBITO_CAT', 'CAUSABAS', 'LOCOCOR_DESC']
df_kmeans = df_merged[variaveis].dropna().copy()

# Codificar variáveis categóricas
label_cols = ['SEXO', 'HORAOBITO_CAT', 'CAUSABAS', 'LOCOCOR_DESC']
for col in label_cols:
    le = LabelEncoder()
    df_kmeans[col] = le.fit_transform(df_kmeans[col].astype(str))

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_kmeans)

# Rodar KMeans com número arbitrário de clusters (ajustamos depois)
kmeans = KMeans(n_clusters=6, random_state=42, n_init='auto')
df_kmeans['cluster'] = kmeans.fit_predict(X_scaled)

# Ver agrupamento básico
df_kmeans['cluster'].value_counts().sort_index()

,count
cluster,
0,9328
1,3945
2,1363
3,1960
4,14144
5,5005


In [ ]:
df_kmeans.groupby('cluster').mean(numeric_only=True)

,IDADE_ANOS,SEXO,HORAOBITO_CAT,CAUSABAS,LOCOCOR_DESC
cluster,,,,,
0,29.017402,0.999678,0.449507,71.851201,1.636792
1,48.345501,1.001267,2.074525,105.298606,1.730798
2,39.503910,2.000000,1.968452,70.852531,3.983859
3,32.316281,2.000000,1.948469,75.716327,1.476531
4,28.654736,0.999859,3.089367,69.862274,1.619132
5,36.605618,0.999800,2.070929,67.670330,3.987013


In [ ]:
df_kmeans.groupby('cluster')[['SEXO', 'HORAOBITO_CAT', 'CAUSABAS', 'LOCOCOR_DESC']].agg(lambda x: x.value_counts().index[0])

,SEXO,HORAOBITO_CAT,CAUSABAS,LOCOCOR_DESC
cluster,,,,
0,1,0,70,2
1,1,3,92,2
2,2,3,88,4
3,2,3,70,2
4,1,3,70,2
5,1,3,66,4


# **Conclusão**

🧩 Conclusão

Neste notebook, foi realizada uma análise exploratória e automatizada sobre registros de óbitos por causas externas no Brasil (SIM/DATASUS), com o objetivo de identificar padrões anômalos ou possíveis indícios de serialidade criminal.
- Foram aplicadas técnicas de:
- Limpeza e transformação de dados brutos .dbc
- Integração com tabelas auxiliares oficiais do IBGE
- Agrupamentos suspeitos por perfis de vítima, local, horário e causa
- Modelagem com KMeans para identificar perfis similares de homicídios

📊 O que descobrimos:

- As principais cidades com padrões repetitivos coincidem com os centros urbanos mais violentos do país — o que aponta para violência sistêmica e não serialidade individual.
- Tentativas de encontrar padrões fora do comum (ex: mulheres 40+, causas raras, cidades pequenas) não mostraram agrupamentos suspeitos consistentes.
- A aplicação de KMeans revelou clusters bem distintos de perfis de homicídios, sendo o cluster 2 particularmente relevante por concentrar mulheres com causas menos comuns — ainda assim, sem repetição espacial/temporal forte.

🧠 O que concluímos:

Embora os dados públicos do SIM sejam poderosos, eles não capturam bem nuances temporais e comportamentais finas típicas de serial killers.
Por outro lado, eles revelam com clareza a padronização da violência urbana brasileira, e permitem construir pipelines de investigação úteis para saúde pública, segurança e pesquisa.

🧪 Próximos passos possíveis:

- Adicionar variáveis temporais mais finas (dia da semana, intervalo entre mortes)
- Enriquecer os dados com boletins de ocorrência ou notícias locais
- Criar um índice de “estranheza” para cada homicídio
- Transformar este projeto em artigo, dashboard ou base para aprendizado de IA forense.